In [ ]:
def linear_kernel(**kwargs):
    def f(x1, x2):
        return np.inner(x1, x2) #x1@x2
    return f

def polynomial_kernel(d, **kwargs):
    def f(x1, x2):
        return (np.inner(x1, x2) + 1) ** d
    return f

def gaussian_kernel(sigma, **kwargs):
    def f(x1, x2):
        distance = np.linalg.norm(x1 - x2) ** 2
        return np.exp(-distance / (2 * (sigma ** 2)))
    return f

In [ ]:
class SVM:
    def __init__(self, C=10, kernel=gaussian_kernel, d=4, sigma=None):
        self.C = C # penalty term
        self.kernel = kernel # Kernel function (linear, polynomial, or Gaussian)
        self.d = d # Degree of the polynomial kernel
        self.sigma = sigma # Constant in the Gaussian kernel
        
        # cache values that will need to be used again
        self.kernel_matrix = None
        self.lagr_multipliers = None
        self.support_vectors = None
        self.support_vector_labels = None
        self.intercept = None
        
    def kernel_1(self, X):
        n_samples, n_features = np.shape(X)

        # Default value for sigma
        if not self.sigma:
            self.sigma = 1 / n_features

        # Compute the kernel matrix
        self.kernel = self.kernel(
            d=self.d,
            sigma=self.sigma)
        
        self.kernel_matrix = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                self.kernel_matrix[i, j] = self.kernel(X[i], X[j])
    
    def intercept_1(self, X, y):
        n_samples, n_features = np.shape(X)
        
        # Quadratic optimization problem
        P = cvxopt.matrix(np.outer(y, y) * self.kernel_matrix, tc='d')
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1, n_samples), tc='d')
        b = cvxopt.matrix(0, tc='d')
        G = cvxopt.matrix(np.vstack((np.identity(n_samples) * -1, np.identity(n_samples))))
        h = cvxopt.matrix(np.vstack((cvxopt.matrix(np.zeros(n_samples)), cvxopt.matrix(np.ones(n_samples) * self.C))))

        # Minimize the quadratic problem
        minimization = cvxopt.solvers.qp(P, q, G, h, A, b)

        # Lagrange multipliers
        lagr_mult = np.ravel(minimization['x'])

        # Support vectors
        self.lagr_multipliers = lagr_mult[lagr_mult > 1e-7]
        self.support_vectors = X[lagr_mult > 1e-7]
        self.support_vector_labels = y[lagr_mult > 1e-7]
        
        # Finally, calculate the intercept using the first support vector
        self.intercept = self.support_vector_labels[0]
        for i in range(len(self.lagr_multipliers)):
            self.intercept = self.intercept - self.lagr_multipliers[i] * self.support_vector_labels[i] * self.kernel(self.support_vectors[i], self.support_vectors[0])

    def fit(self, X, y):
        # Fit the training data on the model
        self.kernel_1(X)
        self.intercept_1(X, y)

    def predict(self, X):
        y_pred = []
    
        for x in X:
            # Predict using the lagrange multipliers, support vectors, kernel, and intercept
            prediction = 0
            for i in range(len(self.lagr_multipliers)):
                prediction += self.lagr_multipliers[i] * self.support_vector_labels[i] * self.kernel(self.support_vectors[i], x)
            
            prediction += self.intercept
            y_pred.append(np.sign(prediction)) # The sign function returns -1 if x < 0, 0 if x==0, 1 if x > 0

        return np.array(y_pred)

In [ ]:
X = X_train.values
y = y_train.values

In [ ]:
svm = SVM(C=10, kernel=linear_kernel)
svm.fit(X, y)
pred = svm.predict(X_test.values)

In [ ]:
print('\nTraining Accuracy: '+ str(np.mean((pred.T == y_test) * 100)) + '%')

In [ ]:
svm = SVM(C=10, kernel=polynomial_kernel, d=2)
svm.fit(X, y)
pred = svm.predict(X_test.values)

In [ ]:
print('\nTraining Accuracy: '+ str(np.mean((pred.T == y_test) * 100)) + '%')

In [ ]:
svm = SVM(C=10, kernel=gaussian_kernel, sigma=8)
svm.fit(X, y)
pred = svm.predict(X_test.values)

In [ ]:
print('\nTraining Accuracy: '+ str(np.mean((pred.T == y_test) * 100)) + '%')